In [138]:
import requests
import json
import pandas as pd
import re
import os 
from dotenv import load_dotenv
import pymysql
from datetime import datetime, timedelta
from pytz import timezone

In [139]:
# Getting API key
load_dotenv("flight_api.env")
api_key = os.getenv("API_KEY")


In [140]:
def create_connection(schema):
    host = "127.0.0.1"
    user = "root"
    load_dotenv("sql_password.env")
    password = os.getenv("SQL")
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

connection_string = create_connection("cities")

In [141]:
def fetch_cities_data(connection_string):
        return pd.read_sql("cities", con=connection_string)
    
cities_df = fetch_cities_data(connection_string)

In [142]:
def fetch_codes(cities_df, api_key):
    
    codes_list = []
    
    for _, city in cities_df.iterrows():
        city_name = city["city_name"]
        city_id = city["city_id"]
        
        url = "https://aerodatabox.p.rapidapi.com/airports/search/term"
        querystring = {"q" : city_name,
                       "limit" : "2"
                      }
        headers = {"x-rapidapi-key" : api_key,
                   "x-rapidapi-host" : "aerodatabox.p.rapidapi.com"
                   }
        response = requests.get(url, headers=headers, params=querystring)
        codes_data = response.json()
        
        for i in range(len(codes_data["items"])):
            code_item = {"city_id" : city_id,
                         "city_name": city_name,
                         "icao" : codes_data["items"][i].get("icao"),
                         "iata" : codes_data["items"][i].get("iata"),
                         "airport" : codes_data["items"][i].get("name")
                        }
            codes_list.append(code_item)
    codes_df = pd.DataFrame(codes_list)
    return codes_df        

In [143]:
codes_df = fetch_codes(cities_df, api_key)
codes_df

,city_id,city_name,icao,iata,airport
0,1,Berlin,EDDB,BER,Berlin Brandenburg
1,1,Berlin,EDDT,TXL,Berlin -Tegel
2,2,Hamburg,EDDH,HAM,Hamburg
3,2,Hamburg,EDHI,XFW,Hamburg -Finkenwerder
4,3,Munich,EDDM,MUC,Munich
5,4,London,CYXU,YXU,London
6,4,London,EGGW,LTN,London Luton
7,5,Prague,LKPR,PRG,Prague Ruzyně
8,6,Istanbul,LTFJ,SAW,Istanbul Sabiha Gökçen
9,6,Istanbul,LTFM,IST,Istanbul


In [ ]:
codes_df.to_sql("air_codes",
              con=connection_string,
              if_exists="append",
              index=False)

In [144]:
def fetch_flight_data(df, api_key):
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))
    flight_items = []
    
    for _, city in df.iterrows():
        times = [["00:00","11:59"],
                 ["12:00","23:59"]]
        
        icao = city["icao"]
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        
            querystring = {"withLeg":"true",
                           "direction":"Arrival",
                           "withCancelled":"false",
                           "withCargo":"false",
                           "withLocation":"false"}
        
            headers = {"x-rapidapi-key": api_key,
                       "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
                       }
            response = requests.request("GET",
                                        url, 
                                        headers=headers,
                                        params=querystring)
           
            if response.status_code == 204:
                print(f"No flight data found for {icao} at {time[0]} - {time[1]}. Skipping...")
                continue 
            try:
                flight_data = response.json()
            except requests.exceptions.JSONDecodeError:
                print(f"Failed to decode JSON for {icao} at {time[0]} - {time[1]}. Response: {response.text}")
                continue

            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
        
            for item in flight_data['arrivals']:
                flight_dict = {"scheduled_time_local" :item["arrival"]["scheduledTime"].get("local", None),
                               "arrival_icao" : icao,
                               "departure_icao" : item["departure"]["airport"].get("icao", None),
                               "departure_airport" : item["departure"]["airport"].get("name", None),
                               "flight_number" : item.get("number", None),
                               "flight_status" : item.get("status", None),
                               "airline" : item.get("airline", {}).get("name", None),
                               "data_retrieved_at": retrieval_time
                               }
                flight_items.append(flight_dict)
            
    flight_df = pd.DataFrame(flight_items)
    flight_df["scheduled_time_local"] = flight_df["scheduled_time_local"].str[:-6]
    flight_df["scheduled_time_local"] = pd.to_datetime(flight_df["scheduled_time_local"])
    flight_df["data_retrieved_at"] = pd.to_datetime(flight_df["data_retrieved_at"])
    
    return flight_df
            

In [145]:
flight_df = fetch_flight_data(codes_df,api_key)
flight_df

No flight data found for EDDT at 00:00 - 11:59. Skipping...
No flight data found for EDDT at 12:00 - 23:59. Skipping...
No flight data found for EDHI at 00:00 - 11:59. Skipping...
No flight data found for EDHI at 12:00 - 23:59. Skipping...


,scheduled_time_local,arrival_icao,departure_icao,departure_airport,flight_number,flight_status,airline,data_retrieved_at
0,2025-03-21 06:35:00,EDDB,HEGN,Hurghada,SM 2960,Expected,Air Cairo,2025-03-20 10:57:29
1,2025-03-21 06:40:00,EDDB,ZBAA,Beijing,HU 489,Expected,Hainan,2025-03-20 10:57:29
2,2025-03-21 06:55:00,EDDB,OTHH,Doha,QR 79,Expected,Qatar,2025-03-20 10:57:29
3,2025-03-21 07:30:00,EDDB,LGTS,Thessaloniki,FR 1147,Expected,Ryanair,2025-03-20 10:57:29
4,2025-03-21 07:35:00,EDDB,LFSB,Bâle/Mulhouse,U2 1185,Expected,easyJet,2025-03-20 10:57:29
...,...,...,...,...,...,...,...,...
2481,2025-03-21 23:55:00,LTFM,EGKK,London,TK 1998,Expected,Turkish,2025-03-20 10:57:32
2482,2025-03-21 23:55:00,LTFM,EBBR,Brussels,TK 1940,Expected,Turkish,2025-03-20 10:57:32
2483,2025-03-21 23:55:00,LTFM,LEBL,Barcelona,TK 1856,Expected,Turkish,2025-03-20 10:57:32
2484,2025-03-21 23:55:00,LTFM,URSS,Sochi,A4 5073,Expected,Azimuth,2025-03-20 10:57:32


In [146]:
flight_df.to_sql("flights",
                if_exists="append",
                con=connection_string,
                index=False)

2486

In [147]:
flight_df

,scheduled_time_local,arrival_icao,departure_icao,departure_airport,flight_number,flight_status,airline,data_retrieved_at
0,2025-03-21 06:35:00,EDDB,HEGN,Hurghada,SM 2960,Expected,Air Cairo,2025-03-20 10:57:29
1,2025-03-21 06:40:00,EDDB,ZBAA,Beijing,HU 489,Expected,Hainan,2025-03-20 10:57:29
2,2025-03-21 06:55:00,EDDB,OTHH,Doha,QR 79,Expected,Qatar,2025-03-20 10:57:29
3,2025-03-21 07:30:00,EDDB,LGTS,Thessaloniki,FR 1147,Expected,Ryanair,2025-03-20 10:57:29
4,2025-03-21 07:35:00,EDDB,LFSB,Bâle/Mulhouse,U2 1185,Expected,easyJet,2025-03-20 10:57:29
...,...,...,...,...,...,...,...,...
2481,2025-03-21 23:55:00,LTFM,EGKK,London,TK 1998,Expected,Turkish,2025-03-20 10:57:32
2482,2025-03-21 23:55:00,LTFM,EBBR,Brussels,TK 1940,Expected,Turkish,2025-03-20 10:57:32
2483,2025-03-21 23:55:00,LTFM,LEBL,Barcelona,TK 1856,Expected,Turkish,2025-03-20 10:57:32
2484,2025-03-21 23:55:00,LTFM,URSS,Sochi,A4 5073,Expected,Azimuth,2025-03-20 10:57:32
